In [33]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
df_prov_tot = pd.read_csv('static/csv/be-covid-provinces-all.csv')
from datetime import datetime, date, timedelta

#print(df_prov_tot)

# Cases in each province

In [35]:
def cases_prov(prov=None):
    df_cases = df_prov_tot

    if prov is not 'All':
        df_cases = df_cases[df_cases['PROVINCE'] == prov]

    df = df_cases.groupby(['DATE']).agg({'CASES':'sum','TESTS_ALL':'sum','TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    x2 = df.index
    y2 = df.CASES

    df.index = pd.to_datetime(df.index)
    dw = df.index.dayofweek
    colors_map = {0: '#fccde5', 1: '#8dd3c7', 2: '#b3de69', 3: '#bebada', 4: '#fb8072', 5: '#fdb462', 6: '#80b1d3',
                  7: 'lightgrey'}
    colors = [colors_map[i] for i in dw]

    
    fig = go.Figure(data=[
        go.Bar(x=x2[-20:], y=y2[-20:], name='bars', legendgroup='group2', showlegend=True, marker_color=colors),
        go.Scatter(x=x2[:-2], y=y2[:-2].rolling(7,center=True).mean(), showlegend=True, name=gettext('avg'),legendgroup='avg 7 days', marker_color="red")
    ])
    
    fig.add_vline(x=(datetime.today()-timedelta(days=4)).strftime('%Y-%m-%d'), line_width=3, line_dash="dash", line_color="green")

    
    
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,max(y2[x2 >=start])])
    fig.update_layout(xaxis_range=[start, (datetime.today()).strftime('%Y-%m-%d')])
    fig.update_layout(template="plotly_white", height=300, margin=dict(l=0, r=0, t=30, b=0),title=prov)

    return fig


def cases_per_provinces():
    prov = ['All','Liège', 'Brussels','Antwerpen','Namur', 'Luxembourg', 'Hainaut' , 'BrabantWallon', 'VlaamsBrabant', 'OostVlaanderen',
        'WestVlaanderen', 'Limburg',]
    n_cols = 2
    n_rows = math.ceil(1.0 * len(prov)/ n_cols)

    idx = 0
    for prov_name in prov:
            fig = cases_prov(prov=prov_name)
            
            fig.show()



cases_per_provinces()

# Positive Rate

In [44]:
def plot_positive_rate():
    """
    plot of the ration cases over testing everyday
    """
    
    df = df_prov_tot.groupby(['DATE']).agg({'CASES':'sum','TESTS_ALL':'sum','TESTS_ALL_POS':'sum','TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})


    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index[:-4], y=100 * df.TESTS_ALL_POS[:-4].rolling(7,center=True).mean() / df.TESTS_ALL.rolling(7, center=True).mean(),
                             mode='lines',
                             name='avg',
                             marker_color="blue", legendgroup="two", showlegend=True))

    fig.add_trace(go.Scatter(x=df.index, y=100 * df.TESTS_ALL_POS[:-4] / df.TESTS_ALL[:-4],
                             mode='markers',
                             name='Pos %',
                             marker_color="blue", legendgroup="twoa", showlegend=True))

    fig.update_layout(xaxis_title='Day',
                      yaxis_title='Pos %', title="Positive rate %")
    fig.update_layout(template="plotly_white")
    
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,15])
    fig.update_layout(xaxis_range=[start, (datetime.today()).strftime('%Y-%m-%d')])

    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig
plot_positive_rate()

# Tests

In [40]:
def plot_tests():
    """
    plot of the ration cases over testing everyday
    """
    
    df = df_prov_tot.groupby(['DATE']).agg({'CASES':'sum','TESTS_ALL':'sum','TESTS_ALL_POS':'sum','TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})


    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index[:-4], y= df.TESTS_ALL.rolling(7, center=True).mean(),
                             mode='lines',
                             name='avg',
                             marker_color="blue", legendgroup="two", showlegend=True))

    fig.add_trace(go.Scatter(x=df.index[:-4], y=df.TESTS_ALL,
                             mode='markers',
                             name='daily',
                             marker_color="blue", legendgroup="twoa", showlegend=True))

    fig.update_layout(xaxis_title=gettext('Day'),
                      yaxis_title=gettext('Number of'), title=gettext("Tests all % (7 days avg)"))
    fig.update_layout(template="plotly_white")
    
    start = '2021-01-01' 
    fig.update_layout(xaxis_range=[start, (datetime.today()).strftime('%Y-%m-%d')])

    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig
plot_tests()